- 이전 섹션에서 모델 로딩과 데이터셋 준비를 마쳤으므로, 이제 재미있는 부분인 DPO 손실 코딩으로 넘어갈 수 있습니다.
- 아래의 DPO 손실 코드는 [Direct Preference Optimization: Your Language Model is Secretly a Reward Model](https://arxiv.org/abs/2305.18290) 논문에서 제안한 방법을 기반으로 합니다.
- 참고로 핵심 DPO 수식은 다음과 같습니다:

<img src="images/llm_from_scratch/dpo/3.webp" width=800px>

- 위 식에서,
  - "기댓값(expected value)" $\mathbb{E}$는 통계 용어로 확률 변수(괄호 안의 표현식)의 평균값을 의미합니다. $-\mathbb{E}$를 최적화하면 모델이 사용자 선호도에 더 잘 부합하게 됩니다.
  - $\pi_{\theta}$ 변수는 소위 정책(policy, 강화 학습에서 차용한 용어)이라 불리며 우리가 최적화하고자 하는 LLM을 나타냅니다. $\pi_{ref}$는 참조(reference) LLM으로, 일반적으로 최적화 전의 원본 LLM입니다 (훈련 시작 시점에는 $\pi_{\theta}$와 $\pi_{ref}$가 보통 동일합니다).
  - $\beta$는 $\pi_{\theta}$와 참조 모델 간의 발산(divergence)을 제어하는 하이퍼파라미터입니다. $\beta$를 높이면 전체 손실 함수에서 $\pi_{\theta}$와 $\pi_{ref}$의 로그 확률 차이가 미치는 영향이 커져서, 두 모델 간의 차이(divergence)가 증가하게 됩니다.
  - 로지스틱 시그모이드 함수 $\sigma(\centerdot)$는 선호되는 응답과 거부되는 응답의 로그 오즈(log-odds, 시그모이드 함수 내부의 항)를 확률 점수로 변환합니다.
- 코드로 DPO 손실을 다음과 같이 구현할 수 있습니다:

In [ ]:
import torch.nn.functional as F

def compute_dpo_loss(
    model_chosen_logprobs,
    model_rejected_logprobs,
    reference_chosen_logprobs,
    reference_rejected_logprobs,
    beta=0.1,
):
    """
    DPO(Direct Preference Optimization) 손실 함수를 계산합니다.
    학습 중인 모델(Policy)이 고정된 기준 모델(Reference)보다
    '선호되는 답변(Chosen)'에 더 높은 확률을 부여하도록 유도합니다.
    """

    # 1. [학습 모델]의 선호도 격차 계산 (Model Log Ratios)
    # 수식: log(Chosen확률) - log(Rejected확률) = log(Chosen확률 / Rejected확률)
    # 의미: 학습 모델이 Rejected보다 Chosen을 얼마나 더 선호하는가?
    # 값이 클수록 정답을 더 강하게 확신한다는 뜻입니다.
    model_logratios = model_chosen_logprobs - model_rejected_logprobs

    # 2. [참조 모델]의 선호도 격차 계산 (Reference Log Ratios)
    # 의미: 원래 모델(기준점)은 Rejected보다 Chosen을 얼마나 더 선호했는가?
    # 이 값은 학습 중에 변하지 않는 기준선(Base line) 역할을 합니다.
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs

    # 3. 두 모델의 격차 비교 (Logits)
    # 수식: (학습 모델의 확신도) - (참조 모델의 확신도)
    # 목적: 참조 모델이 확신하는 정도보다 '더' 정답을 확신하게 만들어야 합니다.
    logits = model_logratios - reference_logratios

    # 4. 최종 Loss 계산 (DPO 공식의 핵심)
    # 공식: -log(sigmoid(beta * logits))
    # 원리:
    #   - logits가 양수(학습 모델이 참조 모델보다 더 잘함) -> sigmoid는 1에 가깝고 -> -log(1)은 0 (Loss 낮음)
    #   - logits가 음수(학습 모델이 못함) -> sigmoid는 0에 가깝고 -> -log(0)은 무한대 (Loss 높음)
    # beta: 학습 강도를 조절하는 하이퍼파라미터 (KL Divergence 제약의 역수 역할)
    losses = -F.logsigmoid(beta * logits)

    # 5. [모니터링용] 암시적 보상 (Implicit Rewards) 계산
    # DPO는 별도의 리워드 모델이 없지만, 수식적으로 리워드를 역산해볼 수 있습니다.
    # 보상 = log(학습모델 확률 / 참조모델 확률) -> 학습 모델이 참조 모델보다 확률을 얼마나 높였는지 측정
    # detach(): 이 계산은 학습(역전파)에는 쓰이지 않고 기록용으로만 쓰므로 그래디언트를 끊습니다.
    chosen_rewards = (model_chosen_logprobs - reference_chosen_logprobs).detach()
    rejected_rewards = (model_rejected_logprobs - reference_rejected_logprobs).detach()

    # 배치 내 모든 샘플의 평균을 반환
    return losses.mean(), chosen_rewards.mean(), rejected_rewards.mean()

- 로그에 익숙하시다면, 위 코드에서 적용된 일반적인 관계식 $\log\left(\frac{a}{b}\right) = \log a - \log b$를 참고하세요.
- 이 점을 염두에 두고 몇 가지 단계를 살펴보겠습니다 (나중에 별도의 함수를 사용하여 `logprobs`를 계산할 것입니다).
- 다음 라인부터 시작해 봅시다.

    ```python
    model_logratios = model_chosen_logprobs - model_rejected_logprobs
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs
    ```

- 위 라인들은 정책 모델과 참조 모델 모두에 대해 선택된(chosen) 샘플과 거부된(rejected) 샘플 간의 로그 확률(로짓) 차이를 계산합니다 (이는 $\log\left(\frac{a}{b}\right) = \log a - \log b$이기 때문입니다):

$$\log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_\theta (y_l \mid x)} \right) \quad \text{and} \quad \log \left( \frac{\pi_{\text{ref}}(y_w \mid x)}{\pi_{\text{ref}}(y_l \mid x)} \right)$$

- 다음으로, `logits = model_logratios - reference_logratios` 코드는 모델의 로그 비율과 참조 모델의 로그 비율 간의 차이를 계산합니다. 즉,

$$\beta \log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_{\text{ref}} (y_w \mid x)} \right)
- \beta \log \left( \frac{\pi_\theta (y_l \mid x)}{\pi_{\text{ref}} (y_l \mid x)} \right)$$


- 마지막으로, `losses = -F.logsigmoid(beta * logits)`는 로그-시그모이드 함수를 사용하여 손실을 계산합니다. 원래 방정식에서 기대값 내부의 항은 다음과 같습니다.

$$\log \sigma \left( \beta \log \left( \frac{\pi_\theta (y_w \mid x)}{\pi_{\text{ref}} (y_w \mid x)} \right)
- \beta \log \left( \frac{\pi_\theta (y_l \mid x)}{\pi_{\text{ref}} (y_l \mid x)} \right) \right)$$

- 위에서는 로그 확률이 이미 계산되었다고 가정했습니다. 이제 위에서 `compute_dpo_loss` 함수에 전달된 로그 확률, 즉 $\pi_\theta (y_w \mid x)$, ${\pi_\theta (y_l \mid x)}$ 등의 값을 계산하는 데 사용할 `compute_logprobs` 함수를 정의해 봅시다.

In [ ]:
def compute_logprobs(logits, labels, selection_mask=None):
    """
    모델의 출력(Logits)과 정답(Labels)을 비교하여,
    모델이 정답 토큰들에 부여한 '로그 확률'의 평균을 계산합니다.

    Args:
        logits: 모델이 뱉어낸 예측값 (아직 확률로 변환 전). Shape: (배치크기, 문장길이, 단어장크기)
        labels: 실제 정답 단어들의 ID. Shape: (배치크기, 문장길이)
        selection_mask: 패딩이나 질문(Prompt) 등 계산에서 제외할 부분을 표시한 마스크.
    """

    # 1. [핵심] 위치 정렬 (Next Token Prediction)
    # LLM은 '현재 단어'를 보고 '다음 단어'를 맞추는 모델입니다.
    # 따라서 입력(Logits)의 t번째 토큰은 정답(Labels)의 t+1번째 토큰을 예측해야 합니다.
    
    # labels[:, 1:]: 첫 번째 토큰은 예측 대상이 아니므로 제외 (오른쪽으로 한 칸 이동 효과)
    labels = labels[:, 1:].clone()
    
    # logits[:, :-1, :]: 마지막 토큰에 대한 예측은 맞춰볼 정답(다음 토큰)이 없으므로 제외
    logits = logits[:, :-1, :]

    # 2. 확률 변환 (Log Softmax)
    # 모델의 날것 점수(Logits)를 확률 분포(Log Probability)로 변환합니다.
    log_probs = F.log_softmax(logits, dim=-1)

    # 3. 정답에 해당하는 확률만 추출 (Gather)
    # 단어장 전체(5만 개 등)에 대한 확률 중, 실제로 등장한 '정답 단어'의 확률만 콕 집어냅니다.
    selected_log_probs = torch.gather(
        input=log_probs,
        dim=-1,
        index=labels.unsqueeze(-1) # 정답 인덱스에 맞춰 차원 확장
    ).squeeze(-1) # 다시 차원 축소 (배치크기, 문장길이-1)

    # 4. 마스킹 및 평균 계산
    if selection_mask is not None:
        # 마스크도 위에서 잘라낸 labels와 길이를 맞추기 위해 앞부분을 자름
        mask = selection_mask[:, 1:].clone()

        # 마스크 적용:
        # 패딩이나 프롬프트(질문) 부분의 로그 확률은 0으로 만들어버려서 합계에 영향이 없게 함
        selected_log_probs = selected_log_probs * mask

        # 평균 계산:
        # (유효한 토큰들의 로그 확률 합) / (유효한 토큰의 개수)
        # sum(-1)은 문장 길이 방향으로 합계를 구함 -> 결과 Shape: (batch_size,)
        avg_log_prob = selected_log_probs.sum(-1) / mask.sum(-1)

        return avg_log_prob

    else:
        # 마스크가 없으면 그냥 단순 평균
        return selected_log_probs.mean(-1)

In [ ]:
import torch

def compute_dpo_loss_batch(batch, policy_model, reference_model, beta):
    """
    데이터 배치 하나를 받아서 DPO 학습을 위한 손실(Loss)과 보상(Reward)을 계산합니다.
    
    과정:
    1. 학습 모델(Policy)로 Chosen/Rejected 데이터의 로그 확률 계산 (그래디언트 필요 O)
    2. 참조 모델(Reference)로 Chosen/Rejected 데이터의 로그 확률 계산 (그래디언트 필요 X)
    3. 위 4가지 값을 공식에 넣어 최종 Loss 산출
    """

    # ==============================================================================
    # 1. 학습 중인 모델 (Policy Model) 평가
    # ==============================================================================
    # 모델이 Chosen(선호) 응답을 생성할 로그 확률 계산
    # policy_model(batch["chosen"]) -> 모델의 Forward 실행 (Logits 출력)
    policy_chosen_log_probas = compute_logprobs(
        logits=policy_model(batch["chosen"]),
        labels=batch["chosen"],
        selection_mask=batch["chosen_mask"] # 질문(Prompt) 부분은 계산에서 제외
    )
    
    # 모델이 Rejected(비선호) 응답을 생성할 로그 확률 계산
    policy_rejected_log_probas = compute_logprobs(
        logits=policy_model(batch["rejected"]),
        labels=batch["rejected"],
        selection_mask=batch["rejected_mask"]
    )
    
    # ==============================================================================
    # 2. 참조 모델 (Reference Model) 평가
    # ==============================================================================
    # 참조 모델은 학습되지 않으므로(Frozen), 그래디언트 계산을 꺼서 메모리와 속도를 아낍니다.
    with torch.no_grad():
        # 참조 모델 입장에서의 Chosen 응답 확률
        ref_chosen_log_probas = compute_logprobs(
            logits=reference_model(batch["chosen"]),
            labels=batch["chosen"],
            selection_mask=batch["chosen_mask"]
        )
        # 참조 모델 입장에서의 Rejected 응답 확률
        ref_rejected_log_probas = compute_logprobs(
            logits=reference_model(batch["rejected"]),
            labels=batch["rejected"],
            selection_mask=batch["rejected_mask"]
        )

    # ==============================================================================
    # 3. DPO 손실 및 보상 계산
    # ==============================================================================
    # 위에서 구한 4가지 확률 값(Policy Chosen/Rejected, Ref Chosen/Rejected)을
    # DPO 공식에 대입하여 최종 Loss를 구합니다.
    loss, chosen_rewards, rejected_rewards = compute_dpo_loss(
        model_chosen_logprobs=policy_chosen_log_probas,
        model_rejected_logprobs=policy_rejected_log_probas,
        reference_chosen_logprobs=ref_chosen_log_probas,
        reference_rejected_logprobs=ref_rejected_log_probas,
        beta=beta # KL 제약 강도 조절 하이퍼파라미터
    )
    
    # loss: 역전파(Backpropagation)에 사용될 값
    # rewards: 학습이 잘 되고 있는지 확인하기 위한 지표(Metrics)
    return loss, chosen_rewards, rejected_rewards